In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from psycopg2 import connect

username = "test_user"
passwd = "test_password"
hostname = "localhost"
db_name = "test_database"


# Establish a connection to the PostgreSQL database
cnx = connect(user=username, password=passwd, host=hostname, database=db_name)

# Create a cursor object
cursor = cnx.cursor()

# SQL query to retrieve data (adjust the query based on your database schema)
query = """
    SELECT
        category,
        AVG(price) AS avg_price,
        COUNT(*) AS num_products
    FROM
        products
    GROUP BY
        category;
"""

# Execute the query and fetch results into a Pandas DataFrame
try:
    cursor.execute(query)
    result = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(result, columns=columns)
except Exception as e:
    print(f"Error: {e}")
    cnx.rollback()
finally:
    cursor.close()

# Data analysis and visualization
if not df.empty:
    print("Data Analysis Results:")
    print(df)

    # Bar chart
    plt.figure(figsize=(10, 6))
    plt.bar(df['category'], df['avg_price'], color='blue')
    plt.xlabel('Category')
    plt.ylabel('Average Price')
    plt.title('Average Price by Category')
    plt.xticks(rotation=45, ha='right')
    plt.show()

# Close the connection
cnx.close()


In [ ]:
cnx = connect(user=username, password=passwd, host=hostname, database=db_name)

# Create a cursor object
cursor = cnx.cursor()

# SQL query to retrieve data (adjust the query based on your database schema)
query = """
    SELECT
        o.orderdate,
        p.category,
        p.productname,
        od.quantity,
        od.price
    FROM
        orders o
        JOIN orderdetails od ON o.ordernumber = od.ordernumber
        JOIN products p ON od.productcode = p.productcode;
"""

# Execute the query and fetch results into a Pandas DataFrame
try:
    cursor.execute(query)
    result = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    df = pd.DataFrame(result, columns=columns)
except Exception as e:
    print(f"Error: {e}")
    cnx.rollback()
finally:
    cursor.close()

# Data analysis and visualization
if not df.empty:
    # Convert 'orderdate' column to datetime type
    df['orderdate'] = pd.to_datetime(df['orderdate'])

    # Calculate total sales per category
    df['total_sales'] = df['quantity'] * df['price']
    category_sales = df.groupby('category')['total_sales'].sum().reset_index()

    print("Total Sales per Category:")
    print(category_sales)

    # Line chart for sales trend over time
    plt.figure(figsize=(12, 6))
    for category in df['category'].unique():
        category_data = df[df['category'] == category]
        plt.plot(category_data['orderdate'], category_data['total_sales'], label=category)

    plt.xlabel('Order Date')
    plt.ylabel('Total Sales')
    plt.title('Sales Trend Over Time by Category')
    plt.legend()
    plt.show()

# Close the connection
cnx.close()